# Gaussian Mixture Models

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

# An Initial Example - 2D Data

Top berkeley students are all trying to get internships at the hottest new Berkeley startups, such as UBear, DropBear, and BearBnB. We suspect that, given the different skills learned at these three companies, students who've worked at each of them will get different grades than students who worked at the others.

We decide to cluster Berkeley startup interns based on their scores in CS 61A and Data 8, and attempt to derive three clusters that might map to interns at the three startups. Run the cell below to see our initial data.

In [ ]:
from sklearn.datasets import make_classification
np.random.seed(1337)
vals = make_classification(n_features=2, 
                           n_informative=2, 
                           n_redundant=0, 
                           n_classes=3, 
                           n_clusters_per_class=1)
X = vals[0] * 4 + [85, 85]
y = vals[1]
colors = [{0:"red",1:"blue",2:"green"}[l] for l in y]
plt.scatter(X.T[0], X.T[1])
plt.title("Scatter plot of CS 61A grade vs. Data 8 grade")
plt.xlabel("CS 61A Grade")
plt.ylabel("Data 8 Grade")

It seems like there are indeed 3 clusters, that may correspond to interns from these 3 companies. Let's try running the EM algorithm to estimate gaussian parameters for these clusters.

### Part 1 - rolling our own

First, let's consider the simplest possible case - two gaussian classes in one dimension. We consider grades in Data 8 of students who interned at UBark and BearBnB, and try to fit gaussian parameters for each class. First, let's plot the data.

In [ ]:
X_simple = X[y != 2].T[1]
y_simple = y[y != 2]
pd.DataFrame({"Data 8 Score": X_simple}).plot(kind="density")
plt.title("Histogram of student scores in Data 8")

Once again, we can see that our data is clearly bimodal, supporting our idea that it's drawn from a mixture of two gaussians. Let's try to find the parameters of the distributions!

### Setup

Recall from [lecture 26](https://drive.google.com/file/d/0Bze55lezLJhIMWhYOGNNLTJKM0k/view) that we treat our data as a mixture of two gaussians,

$$\pi\mathcal{N}(\mu_1, \sigma_1^2) + (1-\pi)\mathcal{N}(\mu_2, \sigma_2^2)$$

That is, each data point is drawn from distribution $\phi_1 \sim \mathcal{N}(\mu_1, \sigma_1^2)$ with probability $\pi$, and from distribution $\phi_2 \sim \mathcal{N}(\mu_2, \sigma_2^2)$ with probability $(1-\pi)$. $\pi$ thus determines our class proportions - $\pi$ is the proportion of data falling into the first class.

Given this definitional background, let's look into how we estimate these parameters.

### E-Step

Recall that, in the E-step, we estimate soft class memberships given our estimates of parameters. Specifically, we can calculate the posterioir probability $Z_i$ - the probability that point $i$ falls into cluster $1$ - for each data point:

$$\hat{\mathit{Z}_i} = \frac{\hat{\pi}\hat{\phi}_1(X_i)}{\hat{\pi}\hat{\phi}_1(X_i) + (1 - \hat{\pi})\hat{\phi}_2(X_i)}$$

Where $\phi_n(X_i)$ represents the value of the pdf of the gaussian distribution $\phi_n$ at point $X_i$. For $\phi_n$, this is

$$\frac{1}{\sqrt{2\sigma_n^2\pi}} e^{\left(-\frac{(x - \mu_n)^2}{2 \sigma_n^2}\right)}$$

From this we can get "soft" class memberships, in the range $(0, 1)$, for each data point.

We have provided an implementation of the E-step in python below.

In [ ]:
def normal_pdf(x, mu, sigma_sq):
    return 1/np.sqrt(2*sigma_sq*np.pi) * np.e**(-(x - mu)**2/(2*sigma_sq))

def est_Zs(x_i, pi, mu_1, mu_2, sigma_1, sigma_2):
    pdf_1 = normal_pdf(x_i, mu_1, sigma_1)
    pdf_2 = normal_pdf(x_i, mu_2, sigma_2)
    return pi*pdf_1/(pi*pdf_1 + (1-pi)*pdf_2)

# Start with a simple guess for the class parameters
Z_simple = est_Zs(X_simple, 0.5, 60, 100, 10, 10)
Z_simple

### M-Step

Now, using these soft class membership scores, we can update our parameters $\hat{\pi}$, $\hat{\mu}_1$, $\hat{\mu}_2$, $\hat{\sigma}_1^2$, $\hat{\sigma}_2^2$.

In [ ]:
def est_pi_hat(Z):
    return np.mean(Z)

def est_u_hat(i, X, Z):
    if i == 0:
        return (np.sum(Z * X)) / np.sum(Z)
    else:
        return (np.sum((1 - Z) * X)) / np.sum(1 - Z)

def est_sigma_hat(i, X, Z, mu_hat):
    if i == 0:
        return (np.sum(Z * (X - mu_hat)**2)) / np.sum(Z)
    else:
        return (np.sum((1 - Z) * (X - mu_hat)**2)) / np.sum(1 - Z)
    
pi_hat = est_pi_hat(Z_simple)
u_hat_1 = est_u_hat(0, X_simple, Z_simple)
u_hat_2 = est_u_hat(1, X_simple, Z_simple)
sigma_hat_1 = est_sigma_hat(0, X_simple, Z_simple, u_hat_1)
sigma_hat_2 = est_sigma_hat(1, X_simple, Z_simple, u_hat_2)

print("Initial values were 0.5, 60, 100, 10, 10")
print("Values after one iteration:")
print(pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2)

### Repeat until satisfied

As with k-means, when should we be satisfied? And does our gratification come in a single lifetime?

Based on the two update steps above, what are stopping criteria would you suggest?

In the next cell, we implement EM with a criterion to stop when the norm $\left|Z_{old}-Z_{new}\right|$ falls below a certain threshold.

In [ ]:
# Initial guess
pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2 = 0.5, 80, 90, 10, 10
Z_simple = est_Zs(X_simple, pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2)
pi_hat = est_pi_hat(Z_simple)
u_hat_1 = est_u_hat(0, X_simple, Z_simple)
u_hat_2 = est_u_hat(1, X_simple, Z_simple)
sigma_hat_1 = est_sigma_hat(0, X_simple, Z_simple, u_hat_1)
sigma_hat_2 = est_sigma_hat(1, X_simple, Z_simple, u_hat_2)
old_Z = Z_simple
Z_simple = est_Zs(X_simple, pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2)

count = 1
while np.linalg.norm(old_Z - Z_simple) > 1e-3:
    count += 1
    pi_hat = est_pi_hat(Z_simple)
    u_hat_1 = est_u_hat(0, X_simple, Z_simple)
    u_hat_2 = est_u_hat(1, X_simple, Z_simple)
    sigma_hat_1 = est_sigma_hat(0, X_simple, Z_simple, u_hat_1)
    sigma_hat_2 = est_sigma_hat(1, X_simple, Z_simple, u_hat_2)
    old_Z = Z_simple
    Z_simple = est_Zs(X_simple, pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2)

print("{} steps to converge".format(count))
print("Calculated parameters:")
print(pi_hat, u_hat_1, u_hat_2, sigma_hat_1, sigma_hat_2)

How well do these distributions model the true values?

In [ ]:
# Close enough for my standards

ax = pd.DataFrame({"Data 8 Score": X_simple}).plot(kind="density")
pd.DataFrame({"Scores for Class 0 Students": X_simple[Z_simple > 0.5]}).plot(kind="density", ax = ax)
pd.DataFrame({"Scores for Class 1 Students": X_simple[Z_simple < 0.5]}).plot(kind="density", ax = ax)
plt.axvline(84.3, color="black")
plt.title("Student scores in Data 8")

Not bad! We've clearly found our two groups. Now, let's try doing this on our full, 2D, 3-class data.

### Part 2 - multiple dimensions

From here on out, we'll use sklearn.

In [ ]:
from sklearn.mixture import GaussianMixture

Using sklearn's GaussianMixture estimator, we can estimate parameters for all classes, in multiple dimensions. Now, for each class, $\mu_i$ is a length-2 vector, and $\sigma^2_i$ is a 2x2 covariance matrix.

In [ ]:
colors = ["green", "blue", "red"]

def make_ellipses(gmm, ax):
    for n, color in enumerate(colors):
        covariances = gmm.covariances_[n][:2, :2]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                  180 + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)

Fitting our model, we can see that it predicts our simple data quite well.

In [ ]:
model = GaussianMixture(3)
model.fit(X)
preds = model.predict(X)
ax = plt.subplot(111)
make_ellipses(model, ax)
plt.scatter(X.T[0], X.T[1], color=[colors[i] for i in preds])
plt.title("GMM predictions and isoprobability contours")

But, let's consider a more complicated case. Now say we still are interested in finding out which startups students interned at - but instead of merely looking at two courses, we have scores in 10 courses availible to us. Let's look at student scores in the first two classes we have availible:

In [ ]:
from sklearn.datasets import make_classification
np.random.seed(1337)
vals = make_classification(n_features=10, 
                           n_informative=10, 
                           n_redundant=0, 
                           n_classes=3, 
                           n_clusters_per_class=1)
X = vals[0] * 4 + np.ones(10)*85
y = vals[1]
plt.scatter(X.T[0], X.T[1])
plt.title("Scatter plot of Stat 134 grade vs. CS 162 grade")
plt.xlabel("Stat 134 grade")
plt.ylabel("CS 162 grade")

The classes aren't clearly visible, at least, not in two dimensions. Training a GMM on the full 100-dimensional dataset gives the following results, which don't line up too well with the true classes.

In [ ]:
model = GaussianMixture(3)
model.fit(X.T[:2].T)
preds = model.predict(X.T[:2].T)
plt.subplot(121)
plt.scatter(X.T[0], X.T[1], color=[colors[i] for i in preds])
plt.title("GMM Predictions")
plt.subplot(122)
plt.scatter(X.T[0], X.T[1], color=[colors[i] for i in y])
plt.title("True Clusters")

The problem is, the underlying data isn't very gaussian. We can remedy this, to an extent, by performing PCA and then training our GMM on the principal components.

In [ ]:
from sklearn.decomposition import PCA

pcomps = PCA()
pcomps.fit(X)
pc_X = pcomps.transform(X)

model = GaussianMixture(3)
model.fit(pc_X.T[:2].T)
preds = model.predict(pc_X.T[:2].T)
plt.subplot(121)
plt.scatter(pc_X.T[0], pc_X.T[1], color=[colors[i] for i in preds])
plt.title("GMM Predictions")
plt.subplot(122)
plt.scatter(pc_X.T[0], pc_X.T[1], color=[colors[i] for i in y])
plt.title("True Clusters")

Much better results!

## Comparion with k-Means

How does EM clustering differ from k-Means?

k-Means:
* Easier to explain and implement.
* Generally simpler to compute.
* Hard assignment of points to a cluster.
* Dependent on cluster center.

EM clustering:
* Soft assignment of points to a cluster.
* Dependent the probability of beloninging to a specific cluster.
* Biased towards spherical (or ellipsoid) groups.
* Better performance with groups that vary in size.